# DATA QUALITY

This notebook performs the data quality assessment for the **SmartPort Delay Risk Scoring** project.

We use the working dataset generated in the setup phase, and we evaluate:

- Structural data issues (shape, duplicates, missing values)
- Timestamp integrity (scheduled vs estimated vs actual)
- Geographic coordinates (lat / lon validity)
- Movement signal quality (sog, cog, hdg)
- Port-related inconsistencies
- Outliers and anomalous behaviors
- Data types and formatting

The goal is to obtain a clean and reliable dataset for feature engineering and modeling.

## IMPORT LIBRARIES

In [2]:
import os
import numpy as np
import pandas as pd

%config IPCompleter.greedy=True

## IMPORT DATASETS

We work with the **full working dataset**, not the sample because anomalies in maritime data (such as invalid coordinates, timestamp errors, AIS glitches, or isolated missing values) are often rare and unevenly distributed. Using only a sample could hide these issues. To ensure a complete and reliable cleaning process, data quality must be performed on the full working dataset.

In [3]:
root = '/Users/rober/'
project = 'smartport-delay-risk-scoring'
path = root + project

work_file = path + '/02_Data/03_Working/work.csv'
df = pd.read_csv(work_file, low_memory=False)

df

,record_id,updated_ts,ship_name,imo,lat,lon,sog,cog,hdg,dep_port,etd_schedule,etd,atd,arr_port,eta_schedule,eta,ata
0,4137,05/04/2018 19:19,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,04/05/2018 21:25,04/05/2018 21:29
1,4138,05/04/2018 19:20,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,04/05/2018 21:25,04/05/2018 21:30
2,4139,05/04/2018 19:21,Star,9364722,59.4462,24.7726,3.7,17,159,EETLL,05/04/2018 19:30,07/04/2018 15:25,2018-04-05 19:21:17,FIHEL,05/04/2018 21:30,04/05/2018 21:26,04/05/2018 21:46
3,4140,05/04/2018 19:22,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,05/04/2018 19:30,07/04/2018 15:29,2018-04-05 19:18:20,EETLL,05/04/2018 21:30,04/05/2018 21:25,04/05/2018 21:32
4,4141,05/04/2018 19:22,Star,9364722,59.4469,24.7728,2.8,354,107,EETLL,05/04/2018 19:30,07/04/2018 15:25,2018-04-05 19:21:17,FIHEL,05/04/2018 21:30,04/05/2018 21:26,04/05/2018 22:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576565,912177,15/03/2019 07:34,Finlandia,9214379,59.9357,24.8750,20.6,197,195,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,NaN,NaN
576566,912178,15/03/2019 07:35,Finlandia,9214379,59.9308,24.8722,20.6,197,196,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,NaN,NaN
576567,912179,15/03/2019 07:36,Finlandia,9214379,59.9242,24.8683,20.7,196,196,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,NaN,NaN
576568,912180,15/03/2019 07:37,Finlandia,9214379,59.9198,24.8657,20.8,197,195,FIHEL,15/03/2019 07:00,15/03/2019 07:00,2019-03-15 06:51:54,EETLL,15/03/2019 09:15,NaN,NaN


## RENAME COLUMNS

We already did it in the setup notebook, remember to recover that part when doing retraining and execution code

## OVERALL CHECK

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576570 entries, 0 to 576569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   record_id     576570 non-null  int64  
 1   updated_ts    576570 non-null  object 
 2   ship_name     576570 non-null  object 
 3   imo           576570 non-null  int64  
 4   lat           576570 non-null  float64
 5   lon           576570 non-null  float64
 6   sog           576570 non-null  float64
 7   cog           576570 non-null  int64  
 8   hdg           576570 non-null  int64  
 9   dep_port      576570 non-null  object 
 10  etd_schedule  576570 non-null  object 
 11  etd           576569 non-null  object 
 12  atd           571904 non-null  object 
 13  arr_port      576569 non-null  object 
 14  eta_schedule  576569 non-null  object 
 15  eta           225545 non-null  object 
 16  ata           225545 non-null  object 
dtypes: float64(3), int64(4), object(10)
memory usage

### Initial observations from the structural check

The overall structure looks consistent with ship-tracking data, but several columns require transformation before we continue with detailed quality checks:

- **Timestamp columns** (`updated_ts`, schedules, ETD/ETA/ATD/ATA) are read as `object`: must be converted to `datetime`
- **Geolocation columns** (`lat`, `lon`) appear numeric, but we will later verify that their ranges fall within valid geographic bounds.
- **Movement variables** (`sog`, `cog`, `hdg`) need distribution inspection to detect impossible or extreme values typical of AIS glitches.
- **Categorical identifiers** (`ship_name`, `dep_port`, `arr_port`) are stored as object/string, which is expected.

Based on these observations, the next steps will focus on:
1. Converting and validating all timestamp fields.  
2. Checking latitude and longitude ranges.  
3. Inspecting movement variables for outliers.  
4. Evaluating missing values and duplicates.

This ensures the dataset is structurally sound before moving into feature engineering and modeling.


## UNIQUE VALUES

In [5]:
df.nunique().sort_values()

arr_port             2
dep_port             2
imo                  4
ship_name            6
sog                293
hdg                361
cog                361
lon               3332
etd_schedule      3984
etd               4307
eta_schedule      4326
atd               6046
lat               7104
eta              19048
ata              46236
updated_ts      305091
record_id       576570
dtype: int64

It looks reasonable:

- Only a few departure and arrival ports: this reflects a limited port-to-port route.
- A small set of vessels (IMO numbers and ship names): typical for controlled datasets.
- Thousands of unique latitude and longitude points: consistent with continuous AIS tracking.
- Movement variables (`sog`, `cog`, `hdg`) show a wide but plausible range of unique values.
- Scheduled and actual timestamps have many unique entries, as expected for individual voyages.
- A large number of `record_id` and `updated_ts` values: each AIS message is a distinct event.

Overall, the structure and uniqueness counts look coherent with a real ship-tracking dataset.

## DATA TYPES

### Convert datetime-like columns

We explicitly convert the timestamp columns to datetime, coercing invalid values to `NaT`. This makes later time-based checks and feature engineering more robust and easier to reason about.

In [6]:
df_cols = [
    "updated_ts", 
    "etd_schedule", 
    "eta_schedule", 
    "etd", 
    "eta", 
    "atd", 
    "ata"
]

for col in df_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

df[df_cols].head()

,updated_ts,etd_schedule,eta_schedule,etd,eta,atd,ata
0,2018-05-04 19:19:00,2018-05-04 19:30:00,2018-05-04 21:30:00,2018-07-04 15:29:00,2018-04-05 21:25:00,2018-04-05 19:18:20,2018-04-05 21:29:00
1,2018-05-04 19:20:00,2018-05-04 19:30:00,2018-05-04 21:30:00,2018-07-04 15:29:00,2018-04-05 21:25:00,2018-04-05 19:18:20,2018-04-05 21:30:00
2,2018-05-04 19:21:00,2018-05-04 19:30:00,2018-05-04 21:30:00,2018-07-04 15:25:00,2018-04-05 21:26:00,2018-04-05 19:21:17,2018-04-05 21:46:00
3,2018-05-04 19:22:00,2018-05-04 19:30:00,2018-05-04 21:30:00,2018-07-04 15:29:00,2018-04-05 21:25:00,2018-04-05 19:18:20,2018-04-05 21:32:00
4,2018-05-04 19:22:00,2018-05-04 19:30:00,2018-05-04 21:30:00,2018-07-04 15:25:00,2018-04-05 21:26:00,2018-04-05 19:21:17,2018-04-05 22:13:00


## MISSING VALUES

We compute missing values per column to understand:
- where information is incomplete
- which fields may need imputation, filtering or special handling

### Identify

In [7]:
missing = df.isna().sum().sort_values(ascending=False)
missing

ata             351036
updated_ts      351029
etd             351025
etd_schedule    351025
eta_schedule    351025
eta             351025
atd               4666
arr_port             1
record_id            0
hdg                  0
cog                  0
sog                  0
lon                  0
lat                  0
imo                  0
ship_name            0
dep_port             0
dtype: int64

The timestamp fields show a large number of missing values because most rows in the dataset represent AIS position messages, not port-call events. AIS telemetry (lat, lon, sog, etc.) is broadcast continuously while a ship is at sea and does not include ETA, ETD, ATA or ATD information. 

Only a small subset of records correspond to departure or arrival events, so:

- `ata` is missing for all rows before the vessel arrives.
- `eta` and scheduled ETA/ETD fields are missing in many AIS updates where this information was not transmitted.
- `atd` is available more often because once a vessel departs, this value remains known for the rest of the voyage.

These missing values are expected and reflect the structure of maritime tracking data rather than data quality issues.

### Handle

Departure and arrival ports are essential for building delay target features.
Rows without port information cannot be used.

We have to remove them.

In [8]:
df = df.dropna(subset=["dep_port", "arr_port"])
df.shape

(576569, 17)

## DUPLICATES

We check for duplicate rows, which are common in AIS retransmissions.

### Identify

In [9]:
df.duplicated().sum()

np.int64(0)

### Handle (future data)

In [10]:
duplicates = df.duplicated().sum()
if duplicates > 0:
    df = df.drop_duplicates()

## GEO COORDINATES VALIDATION

Latitude must be between -90 and 90.  
Longitude must be between -180 and 180.

### Identify

In [11]:
df[["lat", "lon"]].describe().T

,count,mean,std,min,25%,50%,75%,max
lat,576569.0,59.800425,0.223782,59.4300,59.6032,59.7994,60.0001,60.1540
lon,576569.0,24.807701,0.084544,22.4062,24.7398,24.8021,24.8957,24.9848


In [12]:
# just to check
invalid_lat = df[(df["lat"] < -90) | (df["lat"] > 90)]
invalid_lon = df[(df["lon"] < -180) | (df["lon"] > 180)]

invalid_lat.shape[0], invalid_lon.shape[0]

(0, 0)

### Handle (future data)

In [13]:
df = df[
    df["lat"].between(-90, 90) &
    df["lon"].between(-180, 180)
]

## MOVEMENT VARIABLES — STRUCTURAL VALIDATION

Here we perform only *structural* checks on SOG (speed over ground), COG (course over ground) and HDG (heading). The goal at this stage is to detect **physically impossible values**, not to analyze distributions or behavior: that belongs to the EDA notebook.

Structural checks verify:

- No negative speeds (`sog < 0` is impossible)
- No NaN values in SOG/COG/HDG fields due to type issues
- No impossible heading values (`hdg` must be between 0–360)
- No impossible COG values (`cog` must be between 0–360)

Distribution analysis, typical ranges (e.g., 0–25 knots), and statistical outliers will be explored later in EDA.

In [14]:
# Check for structurally invalid SOG (speed over ground)
invalid_sog = df[(df["sog"] < 0) | (df["sog"] > 70)]  # 70 knots = upper physical limit for large vessels

# Check for structurally invalid COG and HDG values
invalid_cog = df[(df["cog"] < 0) | (df["cog"] > 360)]
invalid_hdg = df[(df["hdg"] < 0) | (df["hdg"] > 360)]

(
    invalid_sog.shape[0],
    invalid_cog.shape[0],
    invalid_hdg.shape[0]
)

(22, 0, 10)

In [15]:
# Remove rows with impossible movement values
df = df[
    (df["sog"].between(0, 70)) &
    (df["cog"].between(0, 360)) &
    (df["hdg"].between(0, 360))
]

## TIMESTAMP CONSISTENCY

We check for structurally impossible time sequences:

- ETA cannot be earlier than ETD.
- ATA cannot be earlier than ATD.

These represent corrupted port-call metadata.


### Identify

In [16]:
timing_issues = df[
    ((df["eta"] < df["etd"]) & df["eta"].notna() & df["etd"].notna()) |
    ((df["ata"] < df["atd"]) & df["ata"].notna() & df["atd"].notna())
]

timing_issues.shape[0]

109053

### Handle

In [17]:
df = df[
    ~(
        ((df["eta"] < df["etd"]) & df["eta"].notna() & df["etd"].notna()) |
        ((df["ata"] < df["atd"]) & df["ata"].notna() & df["atd"].notna())
    )
]

## SPLIT NUMERICAL / CATEGORICAL

We now separate numerical and categorical variables.  

This does **not** modify the dataset, it's metadata used for:

- guiding EDA  
- choosing feature engineering strategies  
- designing preprocessing pipelines  

After grouping, we will run basic Data Quality checks for each group:
- Null values (impact assessment)
- Unique values (categorical)
- Simple structural outlier indicators (numerical)

In [18]:
num_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

num_cols, cat_cols

(['record_id', 'imo', 'lat', 'lon', 'sog', 'cog', 'hdg'],
 ['ship_name', 'dep_port', 'arr_port'])

### CATEGORICAL

For each categorical variable we inspect:

- Null values
- Number of unique categories
- Presence of potentially invalid or rare categories (only flagged here)

No transformations are applied here; decisions will be made in EDA or Feature Engineering.

#### Null values, unique values

In [19]:
cat_summary = pd.DataFrame({
    "nulls": df[cat_cols].isna().sum(),
    "unique_values": df[cat_cols].nunique()
})

cat_summary

,nulls,unique_values
ship_name,0,6
dep_port,0,2
arr_port,0,2


#### Categories with very low frequency

In [20]:
rare_categories = {}

for col in cat_cols:
    value_counts = df[col].value_counts(dropna=True, normalize=True)
    rare = value_counts[value_counts < 0.001]  # <0.1% frequency
    if len(rare) > 0:
        rare_categories[col] = rare

rare_categories

{}

### NUMERICAL

We inspect:
- Null values
- Basic descriptive statistics
- Simple outlier indicators (not for removal)

#### Nulls

In [21]:
num_nulls = df[num_cols].isna().sum().sort_values(ascending=False)
num_nulls

record_id    0
imo          0
lat          0
lon          0
sog          0
cog          0
hdg          0
dtype: int64

#### Stats overview

In [22]:
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
record_id,467484.0,4.528955e+05,249407.704081,2.040400e+04,2.354808e+05,4.547060e+05,6.496772e+05,9.121830e+05
imo,467484.0,9.399488e+06,276196.623352,8.919805e+06,9.214379e+06,9.364722e+06,9.773064e+06,9.773064e+06
lat,467484.0,5.980032e+01,0.223818,5.944380e+01,5.960300e+01,5.979920e+01,6.000010e+01,6.015400e+01
lon,467484.0,2.480802e+01,0.080939,2.460040e+01,2.473990e+01,2.480220e+01,2.489570e+01,2.498480e+01
sog,467484.0,1.925826e+01,5.927118,0.000000e+00,1.650000e+01,2.180000e+01,2.280000e+01,4.880000e+01
cog,467484.0,1.492712e+02,111.187125,0.000000e+00,2.200000e+01,1.800000e+02,2.020000e+02,3.600000e+02
hdg,467484.0,1.567985e+02,110.208007,0.000000e+00,2.300000e+01,1.830000e+02,2.040000e+02,3.590000e+02


#### Outliers

In [23]:
outlier_flags = {}

for col in num_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    count = df[(df[col] < lower) | (df[col] > upper)].shape[0]
    if count > 0:
        outlier_flags[col] = int(count)

outlier_flags

{'sog': 37746}

### Interpretation

The IQR rule detects a large number of statistical outliers in `sog` (Speed Over Ground). 
This is expected because vessel speeds are not normally distributed and often contain legitimate extreme values (e.g., acceleration, deceleration, maneuvering near ports, or
sensor noise).

These outliers are **not removed here** because:

- The IQR rule indicates *statistical* extremes, not *invalid* values.
- `sog` already passed the domain-validity check (0–70 knots), so all values are physically plausible.
- Decisions about outlier handling belong to **EDA / Feature Engineering**, not tostructural data quality.

We simply flag the presence of statistical outliers, which will be examined later in EDA.

## CONCLUSION

## 11. Conclusion

The dataset has been fully validated and is now structurally ready for EDA and feature engineering. All integrity checks were performed, including:

- **Type validation:** All timestamp fields converted to proper datetime format.
- **Duplicate inspection:** No duplicates found, but a deduplication rule will be kept in the preprocessing pipeline to protect against future ingestion issues.
- **Domain validity checks:** 
  - Latitude, longitude, SOG, COG, and HDG values fall within physically valid ranges.
  - Timestamp sequences (ETD/ETA/ATD/ATA) respect logical ordering.
- **Missing values:** Present mainly in ETA/ETD/ATA fields, which is expected for AIS data.
- **Categorical variables:** Inspected for nulls and rare categories (flagged only).
- **Numerical variables:** Nulls and statistical outliers identified (but not removed), as they represent behavioral patterns rather than structural errors.
- **Metadata saved:** Numerical and categorical column lists exported for consistent use in EDA and downstream preprocessing.

No structural issues remain, and the cleaned dataset (`work_clean.csv`) is ready for exploratory data analysis, where we will study variable distributions, temporal patterns, route behavior, and begin deriving features for the SmartPort delay-risk scoring model.

## SAVE DATASETS AFTER DATA QUALITY

Save the progress in **df**, **cat**, and **num** with a suffix so you can save or retrieve the project’s progress.

In **pickle** format so as not to lose the metadata modifications.

## SAVE CLEANED DATASET + METADATA

We now save:
- the cleaned dataset
- numerical column list
- categorical column list

These objects will be reused in EDA and Feature Engineering.

In [25]:
# Paths
path_clean = path + '/02_Data/03_Working/work_clean.csv'
path_num_cols = path + '/02_Data/03_Working/num_cols.csv'
path_cat_cols = path + '/02_Data/03_Working/cat_cols.csv'

# Save cleaned dataset
df.to_csv(path_clean, index=False)

# Save metadata lists
pd.Series(num_cols).to_csv(path_num_cols, index=False, header=False)
pd.Series(cat_cols).to_csv(path_cat_cols, index=False, header=False)

path_clean, path_num_cols, path_cat_cols

('/Users/rober/smartport-delay-risk-scoring/02_Data/03_Working/work_clean.csv',
 '/Users/rober/smartport-delay-risk-scoring/02_Data/03_Working/num_cols.csv',
 '/Users/rober/smartport-delay-risk-scoring/02_Data/03_Working/cat_cols.csv')